In [18]:
#First look after first cleaning
import sys
print(sys.executable)

import pandas as pd
print(pd.__version__)

# Load the dataset
file_path = 'data/properties06191148_cleaned.csv'
data = pd.read_csv(file_path)

# Data Overview
print("Data Overview:")
print(f"Rows: {data.shape[0]}")
print(f"Columns: {data.shape[1]}")
print("\nData types:")
print(data.dtypes)

# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing Values:")
print(missing_values)

# Check for duplicates
duplicates = data.duplicated().sum()
print(f"\nDuplicates: {duplicates}")

# Display basic info about the dataset
print("\nBasic Information:")
print(data.info())

# Show the first few rows of the dataset to inspect the data
print("\nFirst few rows of the dataset:")
print(data.head())

# Count the number of "Unknown" values in the 'Region' column
unknown_count = data[data['Region'] == 'Unknown'].shape[0]

print(f"Number of 'Unknown' values in the 'Region' column: {unknown_count}")



/Users/Marc/Documents/GitHub/immo-eliza-lions/.venv/bin/python


ModuleNotFoundError: No module named 'pandas'